### Replicación de cubos de S3 para cumplir con los objetivos del punto de recuperación

#### Problema
La política de seguridad de datos de su empresa exige que los objetos se repliquen dentro de la misma región para cumplir un objetivo de punto de recuperación de 15 minutos.

#### Solución
En primer lugar, cree cubos S3 de origen y destino con el control de versiones activado. A continuación, cree una función de IAM y adjunte una política de IAM que permita a S3 copiar objetos del cubo de origen al de destino. Por último, cree una política de replicación de S3 que haga referencia a la función de IAM y aplique esa política al bucket de origen.
<br>
<br>
<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0303.png" width="400">

In [12]:
import boto3
import json
import random
from pprint import pprint 

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

iam = boto3.resource('iam')
iam_client = boto3.client('iam')

In [3]:
# Create a bucket
bucket_name = 'cookbook-{}'.format(random.randint(10000, 1000000))
bucket = s3.create_bucket(Bucket=bucket_name)
print("Bucket name: {}".format(bucket_name))

# Por defecto cuando creamos un bucket, las opciones de **Block all public access** (Bloquear todo el acceso público) 
# están desactivadas por defecto.

response = s3_client.put_public_access_block(
        Bucket=bucket_name,
        PublicAccessBlockConfiguration={
            'BlockPublicAcls': True,
            'IgnorePublicAcls': True,
            'BlockPublicPolicy': True,
            'RestrictPublicBuckets': True
        }
)

Bucket name: cookbook-143157


In [9]:
# Activate versioning enabled in the bucket:
response = s3_client.put_bucket_versioning(
    Bucket=bucket_name,
    VersioningConfiguration={
        'Status': 'Enabled'
    }
)

if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print("Bucket versioning enabled")
else:
    print("Bucket versioning not enabled")

Bucket versioning enabled


#### Cree el bucket S3 de destino

In [6]:
# Create a bucket
bucket_dest_name = 'cookbook-dest-{}'.format(random.randint(10000, 1000000))
bucket = s3.create_bucket(Bucket=bucket_dest_name)
print("Bucket name: {}".format(bucket_dest_name))

# Por defecto cuando creamos un bucket, las opciones de **Block all public access** (Bloquear todo el acceso público) 
# están desactivadas por defecto.

response = s3_client.put_public_access_block(
        Bucket=bucket_dest_name,
        PublicAccessBlockConfiguration={
            'BlockPublicAcls': True,
            'IgnorePublicAcls': True,
            'BlockPublicPolicy': True,
            'RestrictPublicBuckets': True
        }
)

Bucket name: cookbook-dest-623494


In [10]:
# Activate versioning enabled in the bucket:
response = s3_client.put_bucket_versioning(
    Bucket=bucket_dest_name,
    VersioningConfiguration={
        'Status': 'Enabled'
    }
)

if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print("Bucket versioning enabled")
else:
    print("Bucket versioning not enabled")

Bucket versioning enabled


In [11]:
assume_role_policy = {
  "Version": "2012-10-17",
  "Statement": [
  {
    "Effect": "Allow",
    "Principal": {
      "Service": "s3.amazonaws.com"
    },
    "Action": "sts:AssumeRole"
  }
  ]
}

In [14]:
# Create an IAM role using the statement in the variable assume_role_policy
role_name = 'Cookbook-S3-role-{}'.format(random.randint(100, 1000))
role = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy),
    Description='Role for cookbook'
)

In [16]:
s3_perms_policy = {
  "Version":"2012-10-17",
  "Statement":[
  {
    "Effect":"Allow",
    "Action":[
    "s3:GetObjectVersionForReplication",
    "s3:GetObjectVersionAcl",
    "s3:GetObjectVersionTagging"
    ],
    "Resource":[
    "arn:aws:s3:::{}/*".format(bucket_name)
    ]
  },
  {
    "Effect":"Allow",
    "Action":[
    "s3:ListBucket",
    "s3:GetReplicationConfiguration"
    ],
    "Resource":[
         "arn:aws:s3:::{}".format(bucket_name)
         ]
    },
    {
    "Effect":"Allow",
    "Action":[
    "s3:ReplicateObject",
    "s3:ReplicateDelete",
    "s3:ReplicateTags",
    "s3:GetObjectVersionTagging"
    ],
    "Resource":"arn:aws:s3:::{}/*".format(bucket_dest_name)
  }
  ]
}

In [17]:
# Attach the policy to the role you just created:
response = iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName='s3-permissions',
    PolicyDocument=json.dumps(s3_perms_policy)
)

In [19]:
# Get the role arn:
role_arn = iam_client.get_role(RoleName=role_name)['Role']['Arn']

In [25]:
s3_replication_policy = {
  "Rules": [
  {
    "Status": "Enabled",
    "Filter": {
      "Prefix": ""
    },
    "Destination": {
      "Bucket": "arn:aws:s3:::{}".format(bucket_dest_name),
      "Metrics": {
        "Status": "Enabled",
        "EventThreshold": {
        "Minutes": 15
        }
      },
      "ReplicationTime": {
        "Status": "Enabled",
        "Time": {
        "Minutes": 15
        }
      }
    },
    "DeleteMarkerReplication": {
      "Status": "Disabled"
    },
    "Priority": 1
  }
  ],
  "Role": role_arn
}


Configurar la política de replicación para el bucket S3 de origen

In [26]:
response = s3_client.put_bucket_replication(
    Bucket=bucket_name,
    ReplicationConfiguration=s3_replication_policy
)

#### Controles de validación

In [29]:
# View the replication configuration for the source bucket:
response = s3_client.get_bucket_replication(
    Bucket=bucket_name
)

pprint(response['ReplicationConfiguration'])

{'Role': 'arn:aws:iam::831466106657:role/Cookbook-S3-role-502',
 'Rules': [{'DeleteMarkerReplication': {'Status': 'Disabled'},
            'Destination': {'Bucket': 'arn:aws:s3:::cookbook-dest-623494',
                            'Metrics': {'EventThreshold': {'Minutes': 15},
                                        'Status': 'Enabled'},
                            'ReplicationTime': {'Status': 'Enabled',
                                                'Time': {'Minutes': 15}}},
            'Filter': {'Prefix': ''},
            'ID': 'ZmI3OGZlMWMtZDAwMi00MTNlLWJkYmUtZWE5YzZlNGQ5YzE1',
            'Priority': 1,
            'Status': 'Enabled'}]}


In [30]:
# Upload an object to the source bucket:
response = s3_client.put_object(
    Bucket=bucket_name,
    Key='test.txt',
    Body='---This is a test---'
)

In [32]:
# View the replication status for the file that you uploaded to the source bucket:
response = s3_client.head_object(
    Bucket=bucket_name,
    Key='test.txt'
)

pprint(response)

{'AcceptRanges': 'bytes',
 'ContentLength': 20,
 'ContentType': 'binary/octet-stream',
 'ETag': '"6fb061ed5ad2c24aae4ee903406606aa"',
 'LastModified': datetime.datetime(2022, 10, 16, 14, 48, 42, tzinfo=tzutc()),
 'Metadata': {},
 'ReplicationStatus': 'COMPLETED',
 'ResponseMetadata': {'HTTPHeaders': {'accept-ranges': 'bytes',
                                      'content-length': '20',
                                      'content-type': 'binary/octet-stream',
                                      'date': 'Sun, 16 Oct 2022 14:51:38 GMT',
                                      'etag': '"6fb061ed5ad2c24aae4ee903406606aa"',
                                      'last-modified': 'Sun, 16 Oct 2022 '
                                                       '14:48:42 GMT',
                                      'server': 'AmazonS3',
                                      'x-amz-id-2': 'rHVXfljxO6WEx2aQlPuW+aIvuxOWm0qKCdr0x8gq2LI4OTNvJKSIZeXVEwdrFHaVw6vz2rbpRWH4x2QMhWU3Sw==',
                    

Vea el estado de la replicación después de algunos minutos y confirme que `ReplicationStatus` es COMPLETED

### Discusión

Si eres un ingeniero, desarrollador o arquitecto que trabaja en AWS, es muy probable que acabes utilizando S3. Es posible que tenga que implementar algún tipo de replicación en S3 para sus aplicaciones; S3 ofrece dos tipos de replicación para satisfacer sus necesidades específicas: Replicación en la misma región (SRR) y Replicación entre regiones (CRR). El tiempo de replicación es un parámetro configurable de S3 Replication Time Control (S3 RTC) y está documentado para cumplir un objetivo de punto de recuperación (RPO) de 15 minutos respaldado por un acuerdo de nivel de servicio (SLA).

SRR utiliza una función IAM, un bucket de origen y de destino, y una configuración de replicación que hace referencia a la función y a los buckets. En esta receta se utiliza SRR para configurar una replicación unidireccional; puede utilizar SRR para facilitar muchos tipos de casos de uso:

* Agregación de registros a un bucket central para su indexación
* Replicación de datos entre entornos de producción y de prueba
* Redundancia de datos conservando los metadatos de los objetos
* Diseño de la redundancia en torno a la soberanía de los datos y los requisitos de cumplimiento.
* Propósitos de copia de seguridad y archivo

CRR utiliza un rol de IAM similar, un bucket de origen y de destino, y una configuración de replicación que hace referencia al rol y a los buckets. Puede utilizar CRR para ampliar las posibilidades de lo que permite SRR:

* Cumplir con los requisitos de almacenamiento y archivo de datos en todas las regiones
* Ubicar conjuntos de datos similares más cerca de sus necesidades regionales de computación y acceso para reducir la latencia

**NOTA:** Los buckets de S3 que tienen control de versiones añaden marcadores a los objetos que se han eliminado. Ambos tipos de replicación de S3 son capaces de replicar los marcadores de eliminación a su cubo de destino si así lo desea. Para obtener más información, consulte el documento de soporte.

